In [6]:
import onnx 
import torch 
import os
from IPython.display import display
from preProcessing import *
import pandas as pd

## Analysing the output results of FINN compiler

### Directory structure

In [7]:
estimates_output_dir = "output_estimates_only"
! ls -l {estimates_output_dir}
! ls -l {estimates_output_dir}/report

total 20
-rw-r--r-- 1 ocousin xilinx_u  900 Aug 30 16:14 auto_folding_config.json
-rw-r--r-- 1 ocousin xilinx_u  788 Aug 30 16:14 build_dataflow.log
drwxr-xr-x 3 ocousin xilinx_u 4096 Aug 30 16:14 intermediate_models
drwxr-xr-x 2 ocousin xilinx_u 4096 Aug 31 13:57 report
-rw-r--r-- 1 ocousin xilinx_u  406 Aug 30 16:14 time_per_step.json
total 24
-rw-r--r-- 1 ocousin xilinx_u 1784 Aug 30 16:14 estimate_layer_config_alternatives.json
-rw-r--r-- 1 ocousin xilinx_u  222 Aug 31 14:04 estimate_layer_cycles.json
-rw-r--r-- 1 ocousin xilinx_u  955 Aug 30 16:14 estimate_layer_resources.json
-rw-r--r-- 1 ocousin xilinx_u  222 Aug 31 14:26 estimate_network_performance.json
-rw-r--r-- 1 ocousin xilinx_u  780 Aug 30 16:14 op_and_param_counts.json
-rw-r--r-- 1 ocousin xilinx_u  301 Aug 31 14:01 test.json


### Reading estimate_network_performance

In [9]:
print("Reading estimate_network_performance.json ")
estimate_network_performance = pd.read_json(estimates_output_dir + '/report/estimate_network_performance.json')
display(estimate_network_performance)

Reading estimate_network_performance.json 


,critical_path_cycles,max_cycles,max_cycles_node_name,estimated_throughput_fps,estimated_latency_ns
0,249,80,MatrixVectorActivation_2,1250000,2490


### Reading estimate_layer_cycles

In [10]:
print("Reading estimate_layer_cycles.json ")
estimate_layer_cycles = pd.read_json(estimates_output_dir + '/report/estimate_layer_cycles.json')
display(estimate_layer_cycles)

Reading estimate_layer_cycles.json 


,MatrixVectorActivation_0,MatrixVectorActivation_1,MatrixVectorActivation_2,MatrixVectorActivation_3,MatrixVectorActivation_4
0,49,20,80,80,20


### Reading estimate_layer_resources

In [45]:
print("Reading estimate_layer_resources.json ")
estimate_layer_resources = pd.read_json(estimates_output_dir + '/report/estimate_layer_resources.json')
display(estimate_layer_resources)

Reading estimate_layer_resources.json 


,MatrixVectorActivation_0,MatrixVectorActivation_1,MatrixVectorActivation_2,MatrixVectorActivation_3,MatrixVectorActivation_4,total
BRAM_18K,1.000000,4.000000,3.000000,3.000000,1.000000,12.0
BRAM_efficiency,0.042535,0.030382,0.115741,0.115741,0.017361,NaN
LUT,20395.000000,17113.000000,17762.000000,17762.000000,543.000000,73575.0
URAM,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
URAM_efficiency,1.000000,1.000000,1.000000,1.000000,1.000000,NaN
DSP,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


### Extract Relevant information

In [66]:
estimate_network_performance = pd.read_json(estimates_output_dir + '/report/estimate_network_performance.json')
estimate_layer_resources = pd.read_json(estimates_output_dir + '/report/estimate_layer_resources.json')
#dict = {'total':[estimate_network_performance.at[0,'estimated_latency_ns']]}

df = pd.DataFrame({'total':[estimate_network_performance.at[0,'estimated_latency_ns']]},index=['estimated_latency_ns'])
print("display df")
display(df)  #OK


# print("display estimate_layer_resources")
# display(estimate_layer_resources)
#pick-up the total col with the title [[]]
df1 = estimate_layer_resources[['total']].copy()

print("display df1")
display(df1)


#result = df.append(total)
result = pd.concat([df1, df])
display(result)



display df


,total
estimated_latency_ns,2490


display df1


,total
BRAM_18K,12.0
BRAM_efficiency,NaN
LUT,73575.0
URAM,0.0
URAM_efficiency,NaN
DSP,0.0


,total
BRAM_18K,12.0
BRAM_efficiency,NaN
LUT,73575.0
URAM,0.0
URAM_efficiency,NaN
DSP,0.0
estimated_latency_ns,2490.0


## Store the results in different file format for annalysis

In [70]:
result.to_csv('./results/resutls.csv')
result.to_excel('./results/resutls.xlsx')
result.to_json(('./results/resutls.json'))
